# Phase 1 - Setting everything up

Open this repository in a GitHub Codespace.
Before you start with anything else, make sure you setup the infrastructure required. Follow the readme file in the root folder to do this!

To start with Phase 1, if not already done run this in the top level folder:

```
pip install -r requirements.txt
```




In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

print(model_name, deployment_name)
print(client)


Found Azure OpenAI API Base Endpoint: https://cog-mdhi5lxwmoqli.openai.azure.com/
gpt-4o gpt-4o


To see if stuff works in general, you can run this.

In [2]:
response = client.chat.completions.create(
    model = model_name,    
    messages = [{"role" : "assistant", "content" : "The one thing I love more than anything else is "}],
)

print(response.choices[0].message.content)

spending quality time with loved ones. There's something incredibly special about creating memories, sharing laughter, and feeling connected with the people who mean the most to us. What brings you the most joy?


This is the object model for receiving questions.

In [5]:
from enum import Enum
from pydantic import BaseModel

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_or_false = "true_or_false"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


# YOUR Mission: 
Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service. 
Ensure the answers provided are correct and in the correct format.



In [19]:

async def ask_question(ask: Ask):
    # """
    # # Ask a question
    # """

    # Send a completion call to generate an answer
    print('Sending a request to openai')
    
    start_phrase =  ask.question
    question_type = ask.type
    response: openai.types.chat.chat_completion.ChatCompletion = None
    
    if question_type == QuestionType.multiple_choice:
        response = client.chat.completions.create(
            model = deployment_name,
            messages = [{"role" : "assistant", "content" : start_phrase}, 
                     { "role" : "system", "content" : "Answer with the correct option w/o index, no bs:"}]
        )
    elif question_type == QuestionType.true_or_false:
        response = client.chat.completions.create(
            model = deployment_name,
            messages = [{"role" : "assistant", "content" : start_phrase}, 
                     { "role" : "system", "content" : "Answer true or false, no bs"}]
        )
    elif question_type == QuestionType.estimation:
        response = client.chat.completions.create(
            model = deployment_name,
            messages = [{"role" : "assistant", "content" : start_phrase}, 
                     { "role" : "system", "content" : "Give me a number, no bs"}]
        )
    else:
        response = client.chat.completions.create(
            model = deployment_name,
            messages = [{"role" : "assistant", "content" : start_phrase}, 
                     { "role" : "system", "content" : "Answer this question:"}]
        )

    print(response.choices[0].message.content)
    print(response)
    answer = Answer(answer=response.choices[0].message.content)
    answer.correlationToken = ask.correlationToken
    answer.promptTokensUsed = response.usage.prompt_tokens
    answer.completionTokensUsed = response.usage.completion_tokens

    return answer

Use this snippet to try your method with several questions.

In [20]:
ask = Ask(question="Which of the following is a color? a. Tree b. Flower c. Green", type=QuestionType.multiple_choice)
answer = await ask_question(ask)
print('Answer:', answer)

ask = Ask(question="Is Yoda a character from the Star Trek universe: True or False?", type=QuestionType.true_or_false)
answer = await ask_question(ask)
print('Answer:', answer)
ask = Ask(question="How many movies are there in 'The Lord of the Rings' trilogy directed by Peter Jackson?", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer)


Sending a request to openai
Green
ChatCompletion(id='chatcmpl-A8NuQqA9lJH8p6IVxFRR0eqrnrLEL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Green', refusal=None, role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1726561894, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=1, prompt_tokens=40, total_tokens=41), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {}}])
Answer: answer='Green' correlationToken=None promptTokensUsed=40 completionTo

# Sample Questions
Sample Questions could look like this. Make sure your answer exactly  matches the required answer.



{
    "id": 3,
    "phase": 1,
    "question": "Which movie features a plot where a young girl named Dorothy is transported to a magical land via a tornado? 1) Cinderella 2) The Wizard of Oz 3) Alice in Wonderland 4) The Little Mermaid",
    "answer": "The Wizard of Oz",
    "type": "multiple_choice"
},
{
    "id": 4,
    "phase": 1,
    "question": "Is Yoda a character from the Star Trek universe: True or False?",
    "answer": false,
    "type": "true_or_false"
},
{
    "id": 5,
    "phase": 1,
    "question": "How many movies are there in 'The Lord of the Rings' trilogy directed by Peter Jackson?",
    "answer": 3,
    "type": "estimation"
}

Make sure you transfer your code changes into main.py (or additional files). 
You can test your app locally using uvicorn. (See Readme.md for details.)

Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase1 $AZURE_ENV_NAME
```


In [9]:
import unittest
from unittest.mock import patch

class TestAskQuestion(unittest.TestCase):
    @patch('notebook_p1.ask_question')
    async def test_movie_question(self, mock_ask_question):
        # Mock the return value of ask_question
        mock_ask_question.return_value = 'The Wizard of Oz'
        
        # Create the Ask object
        ask = Ask(question="Which movie features a plot where a young girl named Dorothy is transported to a magical land via a tornado? 1) Cinderella 2) The Wizard of Oz 3) Alice in Wonderland 4) The Little Mermaid", type=QuestionType.multiple_choice)
        
        # Call the function and capture the output
        answer = await ask_question(ask)
        self.assertEqual(answer, 'The Wizard of Oz')
        
        # Print the answer (for demonstration purposes)
        print('Answer:', answer)

    @patch('notebook_p1.ask_question')
    async def test_yoda_question(self, mock_ask_question):
        # Mock the return value of ask_question
        mock_ask_question.return_value = False
        
        # Create the Ask object
        ask = Ask(question="Is Yoda a character from the Star Trek universe: True or False?", type=QuestionType.true_or_false)
        
        # Call the function and capture the output
        answer = await ask_question(ask)
        self.assertEqual(answer, False)
        
        # Print the answer (for demonstration purposes)
        print('Answer:', answer)

    @patch('notebook_p1.ask_question')
    async def test_lotr_question(self, mock_ask_question):
        # Mock the return value of ask_question
        mock_ask_question.return_value = 3
        
        # Create the Ask object
        ask = Ask(question="How many movies are there in 'The Lord of the Rings' trilogy directed by Peter Jackson?", type=QuestionType.estimation)
        
        # Call the function and capture the output
        answer = await ask_question(ask)
        self.assertEqual(answer, 3)
        
        # Print the answer (for demonstration purposes)
        print('Answer:', answer)